# 🚀 Advanced SQL

This notebook covers:
- CTEs (Common Table Expressions)
- JOINs
- Window functions
- Aggregate functions
- CASE statements

In [1]:
import snowflake.connector
from snowduck import start_patch_snowflake, seed_table
import pandas as pd

start_patch_snowflake(db_file="examples.duckdb", reset=True)

In [2]:
# Setup test data
employees = {
    "id": [1, 2, 3, 4, 5, 6],
    "name": ["Alice", "Bob", "Carol", "David", "Eve", "Frank"],
    "department": ["Engineering", "Sales", "Engineering", "Marketing", "Engineering", "Sales"],
    "salary": [95000, 75000, 105000, 68000, 98000, 82000],
    "hire_date": pd.to_datetime(["2020-01-15", "2019-03-22", "2018-07-10", "2021-09-01", "2020-11-20", "2019-05-14"]),
}

with snowflake.connector.connect() as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS demo")
    cursor.execute("USE DATABASE demo")
    seed_table(conn, "employees", employees)
    print("Test data ready")

Test data ready


## CASE Statements

In [3]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            name,
            salary,
            CASE 
                WHEN salary >= 100000 THEN 'Senior'
                WHEN salary >= 80000 THEN 'Mid-Level'
                ELSE 'Junior'
            END as level
        FROM employees
        ORDER BY salary DESC
    """)
    
    print(f"{'Name':<10} {'Salary':>10} {'Level':<10}")
    print("-" * 32)
    for row in cursor.fetchall():
        print(f"{row[0]:<10} ${row[1]:>9,} {row[2]:<10}")

Name           Salary Level     
--------------------------------
Carol      $  105,000 Senior    
Eve        $   98,000 Mid-Level 
Alice      $   95,000 Mid-Level 
Frank      $   82,000 Mid-Level 
Bob        $   75,000 Junior    
David      $   68,000 Junior    


## CTEs (Common Table Expressions)

In [4]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        WITH dept_stats AS (
            SELECT 
                department,
                COUNT(*) as emp_count,
                AVG(salary) as avg_salary,
                MAX(salary) as max_salary
            FROM employees
            GROUP BY department
        )
        SELECT * FROM dept_stats
        ORDER BY avg_salary DESC
    """)
    
    print("Department Statistics:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]} employees, avg ${row[2]:,.0f}, max ${row[3]:,}")

Department Statistics:
  Engineering: 3 employees, avg $99,333, max $105,000
  Sales: 2 employees, avg $78,500, max $82,000
  Marketing: 1 employees, avg $68,000, max $68,000


In [5]:
# Multiple CTEs
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        WITH 
            dept_avg AS (
                SELECT department, AVG(salary) as avg_salary
                FROM employees
                GROUP BY department
            ),
            above_avg AS (
                SELECT e.name, e.department, e.salary, d.avg_salary
                FROM employees e
                JOIN dept_avg d ON e.department = d.department
                WHERE e.salary > d.avg_salary
            )
        SELECT * FROM above_avg
        ORDER BY salary DESC
    """)
    
    print("Employees above department average:")
    for row in cursor.fetchall():
        print(f"  {row[0]} ({row[1]}): ${row[2]:,} (avg: ${row[3]:,.0f})")

Employees above department average:
  Carol (Engineering): $105,000 (avg: $99,333)
  Frank (Sales): $82,000 (avg: $78,500)


## Window Functions

In [6]:
# ROW_NUMBER, RANK, DENSE_RANK
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            name,
            salary,
            ROW_NUMBER() OVER (ORDER BY salary DESC) as row_num,
            RANK() OVER (ORDER BY salary DESC) as rank,
            DENSE_RANK() OVER (ORDER BY salary DESC) as dense_rank
        FROM employees
    """)
    
    print(f"{'Name':<10} {'Salary':>10} {'Row#':>5} {'Rank':>5} {'Dense':>5}")
    print("-" * 40)
    for row in cursor.fetchall():
        print(f"{row[0]:<10} ${row[1]:>9,} {row[2]:>5} {row[3]:>5} {row[4]:>5}")

Name           Salary  Row#  Rank Dense
----------------------------------------
Carol      $  105,000     1     1     1
Eve        $   98,000     2     2     2
Alice      $   95,000     3     3     3
Frank      $   82,000     4     4     4
Bob        $   75,000     5     5     5
David      $   68,000     6     6     6


In [7]:
# PARTITION BY - Rank within each department
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            department,
            name,
            salary,
            RANK() OVER (PARTITION BY department ORDER BY salary DESC) as dept_rank
        FROM employees
        ORDER BY department, dept_rank
    """)
    
    print("Salary rankings by department:")
    current_dept = None
    for row in cursor.fetchall():
        if row[0] != current_dept:
            current_dept = row[0]
            print(f"\n{current_dept}:")
        print(f"  #{row[3]}: {row[1]} - ${row[2]:,}")

Salary rankings by department:

Engineering:
  #1: Carol - $105,000
  #2: Eve - $98,000
  #3: Alice - $95,000

Marketing:
  #1: David - $68,000

Sales:
  #1: Frank - $82,000
  #2: Bob - $75,000


In [8]:
# LEAD and LAG
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            name,
            salary,
            LAG(salary) OVER (ORDER BY salary) as prev_salary,
            LEAD(salary) OVER (ORDER BY salary) as next_salary,
            salary - LAG(salary) OVER (ORDER BY salary) as diff_from_prev
        FROM employees
        ORDER BY salary
    """)
    
    print(f"{'Name':<10} {'Salary':>10} {'Prev':>10} {'Next':>10} {'Diff':>10}")
    print("-" * 54)
    for row in cursor.fetchall():
        prev = f"${row[2]:,}" if row[2] else "N/A"
        next_s = f"${row[3]:,}" if row[3] else "N/A"
        diff = f"+${row[4]:,}" if row[4] else "N/A"
        print(f"{row[0]:<10} ${row[1]:>9,} {prev:>10} {next_s:>10} {diff:>10}")

Name           Salary       Prev       Next       Diff
------------------------------------------------------
David      $   68,000        N/A    $75,000        N/A
Bob        $   75,000    $68,000    $82,000    +$7,000
Frank      $   82,000    $75,000    $95,000    +$7,000
Alice      $   95,000    $82,000    $98,000   +$13,000
Eve        $   98,000    $95,000   $105,000    +$3,000
Carol      $  105,000    $98,000        N/A    +$7,000


## Aggregate Functions

In [9]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            COUNT(*) as total,
            SUM(salary) as total_salary,
            AVG(salary) as avg_salary,
            MIN(salary) as min_salary,
            MAX(salary) as max_salary,
            MEDIAN(salary) as median_salary
        FROM employees
    """)
    
    row = cursor.fetchone()
    print("Summary Statistics:")
    print(f"  Count: {row[0]}")
    print(f"  Total: ${row[1]:,}")
    print(f"  Average: ${row[2]:,.0f}")
    print(f"  Min: ${row[3]:,}")
    print(f"  Max: ${row[4]:,}")
    print(f"  Median: ${row[5]:,}")

Summary Statistics:
  Count: 6
  Total: $523,000
  Average: $87,167
  Min: $68,000
  Max: $105,000
  Median: $88,500.0


In [10]:
# LISTAGG - Concatenate values
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            department,
            LISTAGG(name, ', ') as employees
        FROM employees
        GROUP BY department
        ORDER BY department
    """)
    
    print("Employees by department:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]}")

Employees by department:
  Engineering: Alice, Carol, Eve
  Marketing: David
  Sales: Bob, Frank


## QUALIFY - Filter Window Results

In [11]:
# Get top earner from each department using QUALIFY
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("""
        SELECT 
            department,
            name,
            salary
        FROM employees
        QUALIFY RANK() OVER (PARTITION BY department ORDER BY salary DESC) = 1
        ORDER BY salary DESC
    """)
    
    print("Top earner in each department:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]} (${row[2]:,})")

Top earner in each department:
  Engineering: Carol ($105,000)
  Sales: Frank ($82,000)
  Marketing: David ($68,000)


## Session Variables

In [12]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    # Set session variables
    cursor.execute("SET min_salary = 80000")
    cursor.execute("SET dept = 'Engineering'")
    
    # Use variables in queries
    cursor.execute("""
        SELECT name, department, salary
        FROM employees
        WHERE salary >= $min_salary
          AND department = $dept
        ORDER BY salary DESC
    """)
    
    print("Engineers earning >= $80,000:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: ${row[2]:,}")

Engineers earning >= $80,000:
  Carol: $105,000
  Eve: $98,000
  Alice: $95,000


## Pandas Integration

In [13]:
# fetch_pandas_all - get results as DataFrame
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("SELECT name, department, salary FROM employees ORDER BY salary DESC")
    
    df = cursor.fetch_pandas_all()
    print("Results as DataFrame:")
    print(df)
    print()
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

Results as DataFrame:
    name   department  salary
0  Carol  Engineering  105000
1    Eve  Engineering   98000
2  Alice  Engineering   95000
3  Frank        Sales   82000
4    Bob        Sales   75000
5  David    Marketing   68000

DataFrame shape: (6, 3)
Columns: ['name', 'department', 'salary']


In [14]:
# fetch_pandas_batches - process large results in chunks
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE demo")
    cursor.execute("SELECT * FROM employees")
    
    print("Processing in batches:")
    for i, batch_df in enumerate(cursor.fetch_pandas_batches(batch_size=2)):
        print(f"  Batch {i+1}: {len(batch_df)} rows")
        print(f"    Names: {batch_df['name'].tolist()}")

Processing in batches:
  Batch 1: 2 rows
    Names: ['Alice', 'Bob']
  Batch 2: 2 rows
    Names: ['Carol', 'David']
  Batch 3: 2 rows
    Names: ['Eve', 'Frank']
